In [1]:
import numpy as np
import pickle
from sklearn.svm import SVR, SVC, LinearSVR, LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from data_generation import get_data, g_0, m_0
from dml_algorithm import dml_ate

In [16]:
with open('opt_params_svm.pkl', 'rb') as pickle_file:
    svm_params_dict_dict = pickle.load(pickle_file)

In [17]:
for svm_params_dict in svm_params_dict_dict.values():
    for name, params in svm_params_dict[0].items():
        print(name, params)

g0 {'C': 100, 'epsilon': 1, 'kernel': 'linear'}
g1 {'C': 3, 'epsilon': 0.3, 'kernel': 'linear'}
m {'C': 10, 'kernel': 'linear'}
g0 {'C': 30, 'epsilon': 0.1, 'kernel': 'rbf'}
g1 {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
m {'C': 100, 'kernel': 'linear'}
g0 {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
g1 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
m {'C': 0.3, 'kernel': 'linear'}
g0 {'C': 10, 'epsilon': 0.1, 'kernel': 'rbf'}
g1 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
m {'C': 1, 'kernel': 'linear'}
g0 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
g1 {'C': 10, 'epsilon': 1, 'kernel': 'rbf'}
m {'C': 0.3, 'kernel': 'rbf'}
g0 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
g1 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
m {'C': 0.3, 'kernel': 'rbf'}
g0 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
g1 {'C': 10, 'epsilon': 0.3, 'kernel': 'rbf'}
m {'C': 0.3, 'kernel': 'rbf'}


In [2]:
rng = np.random.default_rng(seed=42)

In [39]:
N = 250
y_data, d_data, x_data = get_data(N, rng)
scaler = StandardScaler()
x_data_stand = scaler.fit_transform(x_data)
y_train, y_test, d_train, d_test, x_train, x_test, x_stand_train, x_stand_test = train_test_split(y_data, d_data, x_data, x_data_stand, test_size=0.2, random_state=42)

In [4]:
svm_model_g = SVR(kernel='rbf', C=10, gamma=0.1, epsilon=0.3)
svm_model_m = SVC(kernel='rbf', C=0.3, gamma=0.1, probability=True, random_state=42)
model_g = SVR(C=5)
model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', C=5))

In [12]:
%%time
model_m.fit(x_train, d_train)
print(model_m.predict_proba(x_test)[:20,1])
svm_model_m.fit(x_train, d_train)
print(svm_model_m.predict_proba(x_test)[:20,1])
svm_model_m.fit(x_stand_train, d_train)
print(svm_model_m.predict_proba(x_stand_test)[:20,1])
print(m_0(x_test[:20]))

[0.38456937 0.29668811 0.71552264 0.58918336 0.2747158  0.20923637
 0.25377555 0.11184498 0.76773187 0.37096323 0.63940539 0.03259064
 0.38925722 0.36313623 0.85867783 0.81157203 0.79418176 0.06324784
 0.07803774 0.16877786]
[0.48227487 0.16788704 0.620533   0.5        0.22759594 0.14922451
 0.36659123 0.08847357 0.78503171 0.42193401 0.52085989 0.02789599
 0.21407548 0.28668994 0.93631161 0.9015542  0.85700693 0.05276546
 0.12375482 0.12635343]
[0.47555146 0.16416537 0.65085444 0.52831439 0.24569667 0.13432872
 0.30985467 0.06133524 0.80124488 0.39327475 0.5        0.02732711
 0.24550819 0.23166104 0.94143587 0.90450627 0.8491508  0.03672477
 0.10135619 0.11854056]
[0.41432494 0.20125831 0.67247884 0.54437789 0.25330398 0.13504793
 0.49047732 0.06063403 0.72030511 0.41878014 0.5894452  0.01221842
 0.26911311 0.21678317 0.89742277 0.92414323 0.77226792 0.04850843
 0.07460196 0.07069339]
CPU times: total: 25.3 s
Wall time: 25.7 s


In [11]:
from sklearn.metrics import mean_squared_error

In [40]:
%%time
model_m.fit(x_train, d_train)
print(mean_squared_error(model_m.predict_proba(x_test)[:,1], m_0(x_test)))
svm_model_m.fit(x_train, d_train)
print(mean_squared_error(svm_model_m.predict_proba(x_test)[:,1], m_0(x_test)))
svm_model_m.fit(x_stand_train, d_train)
print(mean_squared_error(svm_model_m.predict_proba(x_stand_test)[:,1], m_0(x_test)))

0.026702033623244567
0.037738635375602556
0.045538710681002785
CPU times: total: 141 ms
Wall time: 136 ms


In [35]:
%%time
model_g.fit(x_train[d_train==0], y_train[d_train==0])
print(mean_squared_error(model_g.predict(x_test), g_0(0, x_test)))
svm_model_g.fit(x_train[d_train==0], y_train[d_train==0])
print(mean_squared_error(svm_model_g.predict(x_test), g_0(0, x_test)))
svm_model_g.fit(x_stand_train[d_train==0], y_train[d_train==0])
print(mean_squared_error(svm_model_g.predict(x_stand_test), g_0(0, x_test)))

1.0763928763707538
0.845974705252724
0.9726778061261214
CPU times: total: 15.6 ms
Wall time: 23.8 ms


In [36]:
%%time
model_g.fit(x_train[d_train==1], y_train[d_train==1])
print(mean_squared_error(model_g.predict(x_test), g_0(1, x_test)))
svm_model_g.fit(x_train[d_train==1], y_train[d_train==1])
print(mean_squared_error(svm_model_g.predict(x_test), g_0(1, x_test)))
svm_model_g.fit(x_stand_train[d_train==1], y_train[d_train==1])
print(mean_squared_error(svm_model_g.predict(x_stand_test), g_0(1, x_test)))

2.091994726884207
1.9550870294240825
2.1874437665098676
CPU times: total: 31.2 ms
Wall time: 18 ms


In [5]:
def svm_cv(y_data, d_data, x_data, cv=5):
    model_g = SVR()
    #model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=5000, random_state=42))
    model_m = SVC(probability=True, random_state=42)
    
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
    }

    grid_search_g = GridSearchCV(estimator=model_g, param_grid=param_grid, cv=cv, n_jobs=-1,
                                 scoring='neg_mean_squared_error')
    grid_search_m = GridSearchCV(estimator=model_m, param_grid=param_grid, cv=cv, n_jobs=-1,
                                 scoring='neg_brier_score')

    svm_params_dict = {}
    for d in [0, 1]:
        grid_search_g.fit(X=x_data[d_data == d], y=y_data[d_data == d])
        svm_params_dict[f'g{d}'] = grid_search_g.best_params_
    grid_search_m.fit(X=x_data, y=d_data)
    svm_params_dict['m'] = grid_search_m.best_params_

    return svm_params_dict

In [12]:
%%time
svm_params_dict = svm_cv(y_data, d_data, x_data)
svm_params_dict

KeyboardInterrupt: 

In [28]:
model_g0, model_g1 = LinearSVR(dual='auto'), LinearSVR(dual='auto')
model_g0_nl, model_g1_nl = SVR(), SVR()
model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto')) #, n_jobs=-1
model_m_nl = SVC(probability=True, random_state=42)

In [52]:
N = 250
y_data, d_data, x_data = get_data(N, rng)
y_train, y_test, d_train, d_test, x_train, x_test = train_test_split(y_data, d_data, x_data, test_size=0.2, random_state=42)
model_m = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=2500))
model_m_2 = CalibratedClassifierCV(estimator=LinearSVC(dual='auto', max_iter=2500), method='isotonic')

In [53]:
%%time
model_m.fit(x_train, d_train)
print(model_m.predict_proba(x_test)[:20,1])
model_m_2.fit(x_train, d_train)
print(model_m_2.predict_proba(x_test)[:20,1])
print(m_0(x_test[:20]))

[0.75782366 0.1081392  0.4380588  0.13226565 0.22477492 0.32252746
 0.52142833 0.55220563 0.3364116  0.61823012 0.76432348 0.29846205
 0.46542094 0.15033277 0.65025305 0.4393908  0.85164284 0.57002081
 0.63447986 0.04004093]
[0.93333333 0.         0.4512605  0.         0.09203475 0.30238095
 0.4512605  0.4912605  0.33952381 0.67239993 0.93333333 0.27507003
 0.4912605  0.05714286 0.72466527 0.4512605  0.96012949 0.54226408
 0.62380952 0.        ]
[0.8485524  0.09058125 0.30348166 0.02835181 0.13868599 0.39836533
 0.46815584 0.49492956 0.23402002 0.82336364 0.70270092 0.35112246
 0.48414538 0.03198229 0.78706381 0.32661086 0.83445733 0.65141855
 0.6412689  0.00732812]
CPU times: total: 109 ms
Wall time: 115 ms


In [ ]:
C, max_iter

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100]
}

In [255]:
%%time
N = 16000
y_data, d_data, x_data = get_data(N, rng)
model_g0, model_g1 = LinearSVR(C=5, dual='auto', max_iter=5000, random_state=42), LinearSVR(C=5, dual='auto', max_iter=5000, random_state=42)
model_g = [model_g0, model_g1]
model_m = CalibratedClassifierCV(estimator=LinearSVC(C=5, dual='auto', max_iter=5000, random_state=42)) #, n_jobs=-1
dml_ate(y_data, d_data, x_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05)

CPU times: total: 3.02 s
Wall time: 3 s


(array([0.51908536, 0.55552273, 0.45258778]),
 5.451014995427961,
 array([0.43462256, 0.60354816]))

In [46]:
%%time
model_g0, model_g1 = LinearSVR(dual='auto', max_iter=2500, random_state=42), LinearSVR(dual='auto', max_iter=2500, random_state=42)
model_g = [model_g0, model_g1]
model_m = SVC(probability=True, random_state=42)
dml_ate(y_data, d_data, x_data, model_g, model_m, K=5, classical=True, inference=True, alpha=0.05)

CPU times: total: 984 ms
Wall time: 986 ms


(array([0.5423967 , 0.64183653, 0.10269771]),
 4.224612415082437,
 array([0.28055733, 0.80423608]))

In [198]:
for N in [250, 500, 1000]:#, 2000, 4000, 8000, 16000]:
    y_data, d_data, x_data = get_data(N, rng)
    svm_params_dict = svm_cv(y_data, d_data, x_data)
    print(svm_params_dict)

{'g0': {'C': 10}, 'g1': {'C': 5}, 'm': {'C': 10}}
{'g0': {'C': 5}, 'g1': {'C': 3}, 'm': {'C': 10}}


C:\Users\henry\MA_CausalML\dml\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'g0': {'C': 10}, 'g1': {'C': 10}, 'm': {'C': 1}}


In [ ]:
choose C=5!